# Aries Basic Controller

The aries basic controller is a simple python library that wraps a swagger api exposed by the BCGov [ACA-Py Agent implementation](https://github.com/hyperledger/aries-cloudagent-python). It is designed to enable easy, programatic interface between business logic of an application (this is what you will write) and the capabilities of the Hyperledger SSI stack currently implemented by this agent codebase, such as the issue-credential protocol. These are all defined in [aries-rfcs](https://github.com/hyperledger/aries-rfcs).

## New to developing with the Hyperledger SSI stack?

Check out this great EdX course - [Becoming a Hyperledger Aries Developer](https://www.edx.org/course/becoming-a-hyperledger-aries-developer). It should give you a good foundation in ACA-Py and the SSI mental model for developers.

If you are short on time and prefer to get stuck into the code in these notebooks we highly recommend you at least run through the [OpenApi lab](https://github.com/cloudcompass/ToIPLabs/blob/master/docs/LFS173x/OpenAPIController.md) from the course.


## Mental Model

Where developing with SSI can be confusing (it was to me at least), is in understanding the mental model. The above courses are great for clarifying this in detail, I will just give a brief overview here.

The components of an SSI application that uses ACA-Py are:

* The business/application logic - This is up to you and your imagination. What credentials does your application issue, what proofs do they expect to verify and what happens once a proof has been successfully verified? It can be as simple of complicated as you want.
* A controller - The aries-basic-contoller is an example of this, but it could be written in any language. It is the mechansism for the business logic to invoke aries protocols within an agent. E.g. write this credential schema to the ledger for me, or issue this connection this credential with these attributes.
* An SSI agent - The implementations of a set of specified aries-rfcs, not all agents have the same set implemented. The agent code interfaces with a ledger, manages a wallet and interacts with other agents.

These set of components make up a single domain, however SSI was developed to simplfy and secure the interaction across domains between SSI applications.

The SSI agent makes connections with other SSI agents and they then go on to exchange messages as part of predefined aries protocols that both understand. SSI agents are event driven. It is the business logic of the applications that determine when to make connections, which protocols to enage in and what responses to communicate.

The general flow of interactions between two SSI applications defined by Alice and Bob is:
* Alice(business logic -> controller -> agent) -> Bob(agent -> controller -> business logic)

We will see this pattern regularly throughout this tutorial. Keep in mind, anything in these notebook is the business logic of the application.

## Docker Setup

Before going any further take a look at the docker-compose.yml file in the root of the [aries-basic-controller tutorial directory](https://github.com/OpenMined/PyDentity/tree/master/tutorials/aries-basic-controller) and try to understand what is going on. 

### Dockerfiles

The PyDentity repo uses common dockerfiles across all tutorials and experiments wherever possible. These can be found within the dockerfiles folder, with subfolders for agents and controller dockerfiles. 


### SSI Applications (Bob and Alice)

The docker-compose.yml defines two separate SSI applications: 
* Bob (bob-agent, bob-notebook) 
* Alice (alice-agent, alice-notebook)

Usually when developing we will simulate at least two separate applications because most interesting aspects of SSI occur during interaction between agents. **Note: For a production deployment you would only develop one application and would expect other production deployments to interact with your agent.** An example of a productionish ssi application template can be found [here](https://github.com/wip-abramson/aries-acapy-fullstack-starter).

The services from Alice and Bob are virtually identical, accept alices agent has an authenticated admin endpoint. More on this later. For simplicity we will just review Alice.

#### alice-agent

All agents us one of the dockerfiles in the dockerfiles/agents folder. To configure and start an agent, a set of arguments must be specified.

Here is a [lab](https://github.com/cloudcompass/ToIPLabs/blob/master/docs/LFS173x/ACA-PyStartup.md) about the aca-py configuration parameters that is part of the EdX course. More details can be found by taking the actual course. 

All enviroment variables such as ALICE_HTTP_PORT are configured in the manage bash script. See lines 119 - 139. As we will see later these variables are used to configure an instance of the AriesAgentController.

We use a common pattern to configure agents throughout this repo. The manage script within a tutorial or project defines the unique arguments for each agent. Within the docker-compose file these arguments are then mapped to generic argument names e.g. HTTP_PORT:${ALICE_HTTP_PORT}. The arguments are then used within a yml file which is passed into the agent on startup. yml files can be found in the configuration folder.

The following command is uesed to populate the yml file and start the aca-py agent using this configuration file.

```Docker
 command: [
        "-c",
          "python3 scripts/parse_yml_env_variables.py -c ./configuration/aries-args-basic.yaml; \
          sleep 20; \
          aca-py start --arg-file /tmp/agent_conf.yml"
      ]
```



#### alice-notebook

This service is created from the defined **Dockerfile**. It installs the aries-basic-controller package from the files within this directory. See lines 11 - 17. In the future this will be a pip installable package. Then it initialises a juypter notebook service that is looking for notebooks in the workspace directory of the container. This directory is populated by mounting a volume in the docker-compose.yml. For example alice-notebook mounts the notebooks/alice notebooks (this is one of them).

Again it is worth pointing out the all code within these notebooks is the business logic of the SSI application.

### Setup

This is a helper service that establishes a connection between Alice and Bob by running the helpers/create_connection/create_connection.py script once the agents have been initialised.

### Hyperledger Indy Ledger Instance

* ledger-browser
* ledger-nodes

This tutorial uses a local ledger consisting of a set of Indy nodes that get destroyed whenever you run ./manage down. It uses the [Von-Network repositiory](https://github.com/bcgov/von-network) to establish this network, more details can be found there. See ledger-nodes service.

Von-Network comes with it's own frontend, this is run through ledger-browser docker service. You can view this by navigating to [http://localhost:9000](http://localhost:9000).

## Aries Basic Controller Library Structure

The library is fairly straightforward. If you have a look into the [aries_basic_controller folder](https://github.com/OpenMined/PyDentity/tree/master/libs/aries-basic-controller/aries_basic_controller). You will see all the code for the library.

The most important file is aries_controller.py, this contains the AriesAgentController class which is the class you will important and instatiate to control a specific ACA-Py instance.

The other files are contained withing three folders:

* **controllers/** - This has a file for each collection of api endpoints. As well as a base.py file which all other controllers extend handling the Restful logic through aiohttp.
* **models/** - This currently only contains connection.py and is used to enable the controller to await a certain state change in a connection. More on this later.
* **helpers/** - Some basic helper functions.

## Instantiating the a AriesAgentController

In [ ]:
# Import the AriesAgentController class from the library
from aries_basic_controller.aries_controller import AriesAgentController

# Define the parameters, these should be the same ones uses to initalise the ACA-Py Agent
# TODO: These should probably be passed in as environment variables generally

# The location the controller spins up a service and listens for webhooks from the agent
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8022
WEBHOOK_BASE = ""

# Configures the controller with the correct endpoint to send api requests too
ADMIN_URL = "http://alice-agent:8021"

# The api key for the admin_url if the agent requires one
API_KEY = "alice_api_123456789"

# Instantiate the AriesAgentController using the correct parameters
agent_controller = AriesAgentController(admin_url=ADMIN_URL, api_key=API_KEY)

## Understanding the Aries Basic Controller Listeners

An instance of AriesAgentController acts as the middle man between the business logic of an application and an ACA-Py agent instance. The business logic can communicate with the ACA-Py instance by sending specific requests to the ADMIN_URL, these requests are implemented in the controllers folder of the aries-basic-controller library.

However, the ACA-Py agent instance also needs to be able to communicate to the business logic of the application. When the agent recieves a message from another agent over a connection, the business logic needs to be able to interpret that message and if needs be respond by communicating through the controller to the agent to take some action.

This is handled through ACA-Py using webhooks. Whenever an ACA-Py agent receives a message, if webhooks are configured (--webhook-url), then generally they post a webhook event identified through the key word topic to the specified endpoint. 

For an instance of AriesBasicController to be listening for these webhook events you must initialise the server by calling:

In [ ]:
agent_controller.init_webhook_server(webhook_host=WEBHOOK_HOST,
                                     webhook_port=WEBHOOK_PORT,
                                     webhook_base=WEBHOOK_BASE)

In [ ]:
await agent_controller.listen_webhooks()

This means that the controller is receiving api requests sent to this endpoint and calling the recieve_webhook function:

```app.add_routes([web.post(self.webhook_base + "/topic/{topic}/", self._receive_webhook)])```

The controller then uses the [PyPubSub](https://pypubsub.readthedocs.io/en/v4.0.3/) library to emit events for every webhook they receive:

```    async def handle_webhook(self, topic, payload):
        logging.debug(f"Handle Webhook - {topic}", payload)
        pub.sendMessage(topic, payload=payload)
        return web.Response(status=200)```

It is up to the business logic of the application (*YOU THE DEVELOPER*) to register listeners for the topic events that your application is interested in and the handler functions specifying how the application will respond to these events.

You can register listeners like this:

In [ ]:
def some_handler(payload):
    print("WRITE YOUR BUSINESS LOGIC HERE")

## The topic must match the topic used by the agent for the webhook.
some_listener = {
    "handler": some_handler,
    "topic": "sometopic"
}

## You can add as many listeners into this array as you want
## The basic controller also specifies some optional default listeners 
## which help update state (see controllers/connection.py) and have some helpful print statements
agent_controller.register_listeners([some_listener], defaults=True)

### You can also add listeners one at a time.

And add more than one listener for the same topic. This is useful in larger applications where multiple processes need to react to a trigger from the agent.

![PyPubSub figure](https://pypubsub.readthedocs.io/en/v4.0.3/_images/pubsub_concept.png)

In [ ]:
def some_handler2(payload):

    print("listener 2")
    # Some other custom function

    
some_listener2 = {
    "handler": some_handler2,
    "topic": "sometopic"
}

agent_controller.add_listener(some_listener2)

### Listeners can be removed individually

In [ ]:
agent_controller.remove_listener(some_listener2)

### And by topic

If topic == None then all listeners will be removed.

In [ ]:
topic = "basicmessages"
agent_controller.remove_all_listeners(topic)



## Terminating the Controller

At the end of all notebook the controller instance must be terminated, so that a new one can be created in following tutorials. If you run into errors around port already in use, you likely forgot to terminate a controller instance from a previous tutorial.

In [ ]:
await agent_controller.terminate()

# Rest of the Tutorial

For the remainder of the tutorial we will be demonstrating some of the common SSI protocols and patterns that you will be regularly use when building SSI applications. These will either involve a single notebook, in which case we use Alice to demonstrate. Or they will involve the interaction between two agents/application Alice and Bob, these are the most interesting examples and will require you to switch between notebooks as you play both parties in the interaction.

These tutorials are largely designed to be stand alone, but if this is your first time exploring this tutorial it is recommended that you follow the order defined.

* Part 3: Establishing a Connection: [Alice](http://localhost:8888/lab/tree/1%20Basic%20Concepts%20and%201st%20Connection/Part%203%20-%20Establishing%20a%20Connection.ipynb), [Bob](http://localhost:8889/lab/tree/1%20Basic%20Concepts%20and%201st%20Connection/Part%203%20-%20Establishing%20a%20Connection.ipynb)
* Part 4: Credential Schema and Definitions: [Alice only](http://localhost:8888/lab/tree/2%20Credentials/Part%201%20-%20Credential%20Schema%20and%20Definitions.ipynb)
* Part 5: Issue Credential: [Alice](http://localhost:8888/lab/tree/2%20Credentials/Part%202%20-%20Issue%20Credential.ipynb), [Bob](http://localhost:8889/lab/tree/2%20Credentials/Part%202%20-%20Issue%20Credential.ipynb)
* Part 6: Present Proof: [Alice](http://localhost:8888/lab/tree/2%20Credentials/Part%203%20-%20Present%20Proof.ipynb), [Bob](http://localhost:8889/lab/tree/2%20Credentials/Part%203%20-%20Present%20Proof.ipynb) (must have completed Part 5)
* Part 7: Basic Messaging: [Alice](http://localhost:8888/lab/tree/3%20Messages/Part%201%20-%20Basic%20Message.ipynb), [Bob](http://localhost:8889/lab/tree/3%20Messages/Part%201%20-%20Basic%20Message.ipynb)
